In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Intro

The first part of this notebook uses a toy dataset to illustrate __how splits are made and evaluated__ when constructing a classification tree.

The example ilustrates splits done on a quantitative predictor (Income)

The example evaluates the splits based on the error rate. Notice that the CART algorithm implemented in scikit learn does not use the error rate to make splits. It uses either the Gini index or Entropy (whichever the user chooses).

__Creating the toy dataset__

There are three predictors. The outcome is Credit_Risk

In [ ]:
toy_df_classif=pd.DataFrame({'Savings':['Med','Low','High','Med','Low','High','Low','Med'], 'Assets':['High','Low','Med','Med','Med','High','Low','Med'], 'Income':[75,50,25,50,100,25,25,75], 'Credit_Risk':['Good','Bad','Bad','Good','Good','Good','Bad','Good']})

In [ ]:
toy_df_classif

In [ ]:
# Error rate at the parent node is the proportion of the least frequent class (which is bad)

3/ 8

#### Exploring how splits are made when constructing a CT
#### Exploring splits on Income

The first thing to do bf creating the splits for a quantitative predictor is to __obtain the cutoff points__

To obtain the possible cutoff points, we first store all Income values (excluding duplicates) in an array __x__

In [ ]:
x= np.sort(toy_df_classif['Income'].unique())
x

In [ ]:
cutpoints=[]
for i in np.arange(len(x)-1):  # I need i to reach only to the second to last index (not the last one)
                               # np.arange(len(x)-1) generates the numbers 0,1,2
    cutpoints.append((x[i]+x[i+1])/2)  

In [ ]:
cutpoints

The algorithm would do the split of Income based on these three cutpoints and would decide the best split (i.e., the split 
leading to the purest child nodes). 

The next code cells will show in details (step by step) how to obtain and evaluate ONLY the split done at 37.5.

Later on, you will see a loop where all three cutoff points are evaluated.

__Reminder__: The algorithm would also do all the possible splits for the other two predictors, Savings and Assets, and would actually choose the best split obtained across all three predictors and across all cutoffs tested for each predictor.

In [ ]:
# LEFT NODE FOR CUTPOINT 37.5

toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ]

# Alternative way: toy_df_classif[toy_df_classif['Income'] < 37.5]

In [ ]:
# RIGHT NODE FOR CUTPOINT 37.5

toy_df_classif.loc[toy_df_classif['Income']>= 37.5, ]

__What's the prediction of Y for the left node? (show students Slide 4)__

Answer: The prediction is Bad because it is the most common class in this node. The next two code cells show you how to get this label programmatically

In [ ]:
toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ]['Credit_Risk'].value_counts()

In [ ]:
toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ]['Credit_Risk'].value_counts().idxmax()

__What's the error rate at the left node?__

Error rate for a given node = 1 - proportion of obs that belong to the predominant class

Error rate for left node = 1- 2/3 (prop of obs that belong to Bad) = 1/3

Altenative to compute the error rate at a node: Direct computation based on proportion of obs that belong to the least frequent class

Error rate for left node = 1/3 (proportion for the least frequent class in this node, which is Good)

__What's the prediction of Y for the right node?__

Using the formula:
Error rate for a given node = 1 - proportion of obs that belong to the predominant class

Error rate for right node = 1- 4/5 (prop of obs that belong to Good)= 0.2

Alternative computation: Direct computation based on proportion of obs that belong to the least frequent class

Error rate for right node = 1/5= 0.2

The next three code cells show you how to get the error rate for the left node using code (based on the direct computation = the proportion for the least frequent class):

In [ ]:
# Obtain the minimum = the proportion for the least frequent class on the left node

min(toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ]['Credit_Risk'].value_counts())

In [ ]:
# Obtain the size of the left node

toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ].shape[0]

In [ ]:
# Obtain the error rate by dividing these two quantities:

min(toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ]['Credit_Risk'].value_counts()) / toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ].shape[0]

Compute the error rate for the right node (based on the direct computation = the proportion for the least frequent class):

In [ ]:
min(toy_df_classif.loc[toy_df_classif['Income'] >= 37.5, ]['Credit_Risk'].value_counts()) / toy_df_classif.loc[toy_df_classif['Income'] >= 37.5, ].shape[0]

__What's the error associated with making a split at 37.5 based on Income?__

Combine the error rate from both the left and right node!

How can we combine these errors? We need to obtain the weighted average of the errors for the left and right nodes

__Error for split at 37.5= weight for left node * error rate for left node + weight for right node * error rate for right node__

weight for left node= obs that reached the left node/ obs before the split

weight for right node= obs that reached the right node/ obs before the split

Note: Notice that the unweighted average (which we do not need to compute), we will be obtained:

Unweighted average = (error rate for left node + error rate for right node)/ 2 = 0.5 * error rate for left node + 0.5 * error rate for right node

In [ ]:
left_node_size= toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ].shape[0]
left_node_size

In [ ]:
right_node_size= toy_df_classif.loc[toy_df_classif['Income'] >= 37.5, ].shape[0]
right_node_size

In [ ]:
# Error at left node

error_left= min(toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ]['Credit_Risk'].value_counts()) / toy_df_classif.loc[toy_df_classif['Income'] < 37.5, ].shape[0]

In [ ]:
# Error at right node

error_right= min(toy_df_classif.loc[toy_df_classif['Income'] >= 37.5, ]['Credit_Risk'].value_counts()) / toy_df_classif.loc[toy_df_classif['Income'] >= 37.5, ].shape[0]

In [ ]:
# Weighted error for the split

error_left * (left_node_size/(left_node_size+right_node_size)) + error_right * (right_node_size/(left_node_size+right_node_size))

The previous five code cells included the steps to compute the weighted error rate for the split at 37.5 on Income.

The next loop summarizes the computation of the weighted error for the three splits on Income, that is the splits at 37.5, 62.5, and 87.5.

We can get what's the best of all the splits for Income based on the output of this loop.

In [ ]:
error_rate_splits =[]
for i in cutpoints:
    left_node_size= toy_df_classif.loc[toy_df_classif['Income'] < i, ].shape[0]
    right_node_size= toy_df_classif.loc[toy_df_classif['Income'] >= i, ].shape[0]
    error_left=min(toy_df_classif['Credit_Risk'][toy_df_classif['Income']<i].value_counts())/left_node_size
    error_right=min(toy_df_classif['Credit_Risk'][toy_df_classif['Income']>=i].value_counts())/right_node_size
    error_rate_splits.append(error_left*(left_node_size/(left_node_size+right_node_size)) + error_right*(right_node_size/(left_node_size+right_node_size)))

In [ ]:
d=pd.DataFrame (data= error_rate_splits, index= cutpoints, columns=['Weighted Error for Split'])

In [ ]:
d.index.name = ('Cutout point')

In [ ]:
d

The best split and only meaningful split is the one at 37.5 because it leads to the lowest error rate (and the only error rate lower than the one for the parent node)

### BACK TO THE SLIDES !

### Example comparing error rate, Gini, and Entropy.

The following example will show that the Gini index and Entropy are more sensitive to changes in the node probabilities than the error rate.

- Y has two classes (0 and 1)
- The node under consideration contains 800 obs, 400 from each class
- Split 1 leads to a left node with 300 zeros and 100 ones AND a right node with  100 zeros and 300 ones.
- Split 2 leads to a left node with 200 zeros and 400 ones AND a right node with  200 zeros and 0 ones.
- Compute the error rate, Gini, and Entropy for both splits.

In [ ]:
# Error rate split 1

# Reminder: Error rate for a split=  Average weighted error 

# Average weighted error= Weight for left node * error rate for left node + weight for right node * error rate for right node

(400/800)* (100/400) + (400/800)* (100/400)


In [ ]:
# Error rate split 2

(600/800)* (200/600) + (200/800)* (0/200) # the right node for split 2 is completely pure!

According to the error rate, splits 1 and 2 are equally good

In [ ]:
# Gini split 1

# Gini index for a split= Weighted Gini for the split

# Weight for left node * Gini for left node + weight for right node * Gini for right nod

# G= 2 p1 (1-p1)


(400/800)* (2*(100/400)*(300/400)) + (400/800)* (2*(300/400)*(100/400))

In [ ]:
# Gini split 2

(600/800)* (2*(400/600)*(200/600)) + (200/800)* (2*(0/200)*(200/200))

According to Gini, split 2 is better. Gini heavily weighs the fact that split 2 gives us a perfectly pure node

In [ ]:
# Entropy split 1

# Entropy for a split= Weighted Entropy for the split

# Weight for left node * Entropy for left node + weight for right node * Entropy for right node

# D= - (1-p1) * log (1 -p1) - p1 * log(p1)

# Reminder: p1 for the left node in split 1 is 100/400
# Reminder: p1 for the right node in split 1 is 300/400

(400/800)* (-(300/400)*np.log(300/400)- (100/400)*np.log(100/400)) + \
(400/800)* (-(100/400)*np.log(100/400) -(300/400)*np.log(300/400))

In [ ]:
# Entropy split 2

# D= - (1-p1) * log (1 -p1) - p1 * log(p1)

# Reminder: p1 for the left node in split 2 is 400/600

(600/800)* (-(200/600)*np.log(200/600)-(400/600)*np.log(400/600)) + (200/800)* 0

# log (0) is undefined. However, when you are in a situation where you are computing the entropy 
# and you get log (0), you can make log(0) equals to 0

According to the Entropy, split 2 is better. Similarly to Gini, Entropy heavily weighs the fact that split 2 gives us a perfectly pure node

### BACK TO THE LAST SLIDES !

<br>
<br>

## Example 1: Obtaining a classification tree for the caravan insurance data 

In [ ]:
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Method needed to apply the GridSearch pre-prunning strategy!

from sklearn.model_selection import GridSearchCV

In [ ]:
Caravan_df= pd.read_csv('C:\\Users\\jheredi2\\Documents\\PythonDataAnalytics\\1-Datasets\\Caravan.csv')

In [ ]:
Caravan_df.info()

In [ ]:
(Caravan_df['Purchase'].value_counts()/Caravan_df['Purchase'].size)*100

In [ ]:
# I chose to exclude the first predictor from the Caravan dataset

X_train, X_test, y_train, y_test= train_test_split (Caravan_df.iloc[:,1:-1], Caravan_df['Purchase'], test_size=0.2, random_state=1)

## Pre-pruning via Grid Search

In [ ]:
hyperparam_grid1 = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1,11),
    'min_samples_split':[0.1, 0.15, 0.2],
    'min_samples_leaf':[0.05, 0.1, 0.15], 
    'min_impurity_decrease': [0, 0.0005, 0.001, 0.005, 0.01, 0.05]
}

In [ ]:
gridSearch1 = GridSearchCV(DecisionTreeClassifier(), hyperparam_grid1, cv=5,scoring='accuracy') 

In [ ]:
gridSearch1.fit(X_train, y_train)

In [ ]:
print('Initial parameters are: ', gridSearch1.best_params_)

In [ ]:
hyperparam_grid2 = {
    'criterion': ['gini'],
    'splitter': ['best'],
    'max_depth': np.arange(1,5),
    'min_samples_split':[0.08, 0.1, 0.12],
    'min_samples_leaf':[0.02, 0.05, 0.08], 
    'min_impurity_decrease': [0, 0.0005, 0.001]
}

In [ ]:
gridSearch2 = GridSearchCV(DecisionTreeClassifier(), hyperparam_grid2, cv=5,scoring='accuracy') 

In [ ]:
gridSearch2.fit(X_train, y_train)

In [ ]:
print('Improved parameters are: ', gridSearch2.best_params_)

In [ ]:
tree_caravan_preprunned= DecisionTreeClassifier(criterion='gini' , splitter= 'best', max_depth= 1, min_samples_split=0.08, min_samples_leaf=0.02, min_impurity_decrease=0, random_state=1)

In [ ]:
tree_caravan_preprunned.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(10,10))   
tree.plot_tree(tree_caravan_preprunned,filled=True, rounded= True, feature_names=X_train.columns, fontsize=12)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
caravan_predicted_test= tree_caravan_preprunned.predict(X_test)

In [ ]:
confusion_matrix(y_test, caravan_predicted_test)

In [ ]:
print (classification_report (y_test, caravan_predicted_test))

## Post-pruning via CCP

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# We are growing the long/unprunned tree using Gini (which is the default criterion)

tree_caravan_unprunned= DecisionTreeClassifier(criterion='gini', random_state=1)

In [ ]:
path= tree_caravan_unprunned.cost_complexity_pruning_path(X_train, y_train)

In [ ]:
alphas= path['ccp_alphas']

In [ ]:
accuracy_scores=[]
for i in alphas:
    treeloop= DecisionTreeClassifier(ccp_alpha=i, random_state=1)
    treeloop.fit(X_train, y_train)
    y_test_predicted=treeloop.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_test_predicted)) 

In [ ]:
indexmax=accuracy_scores.index(max(accuracy_scores))

In [ ]:
tree_caravan_postprunned= DecisionTreeClassifier(ccp_alpha= alphas[indexmax], random_state=1)

In [ ]:
tree_caravan_postprunned.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(20,20))   
tree.plot_tree(tree_caravan_postprunned,filled=True, rounded= True, feature_names=X_train.columns, fontsize=12)
plt.show()

In [ ]:
caravan_postprunned_predicted_test= tree_caravan_postprunned.predict(X_test)

In [ ]:
confusion_matrix(y_test, caravan_postprunned_predicted_test)

In [ ]:
print (classification_report (y_test, caravan_postprunned_predicted_test))

## Example 2: Obtaining a classification tree for the Default data set

## ONLY apply post-pruning via CCP

You will have 10 minutes to work on this task